In [ ]:
#@title Installing needed dependencies!
from IPython.display import clear_output
try:
    from app_store_scraper import AppStore
    from pprint import pprint
    import pandas as pd
    import numpy as np
    import json
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install google-play-scraper
    clear_output()
    %pip install app-store-scraper
    clear_output()
    %pip install google-search-scraper-python
    clear_output()
    %pip install pandas
    clear_output()
    print("All modules required were sucessfully installed!")

In [ ]:
#@title App Reviews from Google Store
from google_play_scraper import app, search, Sort, reviews, reviews_all
from pprint import pprint
import pandas as pd
import numpy as np
import json
#static variables:
language = 'en'
countries = 'us'
appid = []

def searches(nameofSearch):
  result = search(
    nameofSearch,
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=3  # defaults to 30 (= Google's maximum)
)
  return result

def getAppId(r):
  for d in r:
    id = (d['appId'])
    # appid.append(id)
    return id

def insertId(g, cont, rating, choice):

  if choice == None:
    if rating == None:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont # defaults to 100
      # defaults to None(means all score)
    )
      return result
    else:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont, # defaults to 100
      filter_score_with=rating# defaults to None(means all score)
    )
      return result
 
  else:
    if rating == None:
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=None
      )
      return result
    else: 
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=rating # defaults to None(means all score)
      )
      return result

def googleplay(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews

def googleplay1(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews 


appfile = open("googleplay-apps.txt", "r")
#appfile = open("C:\\Users\\nhant\\Research\\apps.txt", "r")
line = appfile.readline()
choice = "Yes" # All reviews
count = None
cond = "No" # No specific reviews
rating = None
while(line):
    app = line

    googleplaydf = pd.DataFrame(np.array(googleplay(app, count, rating, choice)),columns=['review'])
    googleplaydf2 = googleplaydf.join(pd.DataFrame(googleplaydf.pop('review').tolist()))
    #display(googleplaydf2)
    googleplaydf2.to_csv('googleplay-review-' + app.strip() + '.csv')

    line = appfile.readline()


In [ ]:
#@title App Reviews from App Store
from app_store_scraper import AppStore
from pprint import pprint
import pandas as pd
import numpy as np
import json

appfile = open("applestore-apps.txt", "r")
line = appfile.readline()
count = 100000

while(line):
    app = line.strip()

    applereviews = AppStore(country = "us", app_name = app)
    applereviews.review(how_many = count)
    applereviews.reviews
    applestoredf = pd.DataFrame(np.array(applereviews.reviews),columns=['review'])
    applestoredf2 = applestoredf.join(pd.DataFrame(applestoredf.pop('review').tolist()))
    
    applestoredf2.to_csv('applestore-review-' + app + '.csv')
    print(app)

    line = appfile.readline()

In [1]:
appfile = open("common-apps.txt", "r")


FileNotFoundError: [Errno 2] No such file or directory: 'common-apps.txt'